In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# open3d
need_open3d=False
try:
    import open3d
except ModuleNotFoundError:
    need_open3d=True

if need_open3d:
    !pip install open3d


# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
import os
import sys
import torch

import cv2
import numpy as np
#import open3d as o3d
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchvision.io import read_image
from typing import List, Union

from pytorch3d.io import load_obj
from pytorch3d.vis.plotly_vis import plot_scene, plot_batch_individually
from pytorch3d.ops import sample_points_from_meshes

from pytorch3d.structures import (
    Meshes,
    Pointclouds,
    packed_to_list
)

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    RasterizationSettings,
    TexturesUV,
    TexturesVertex
)

from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_normal_consistency,
    mesh_laplacian_smoothing,
    point_mesh_edge_distance,
    point_mesh_face_distance
)

In [ ]:
### Setup
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
else:
    device = torch.device('cpu')

In [ ]:
### Download data for subject 1

import os
import zipfile
import urllib.request as request

attributes = ['body_texture', 'pointcloud'] # ['body', 'body_texture', 'pointcloud']
pointcloud_subjects = [[1, 80], [81, 140], [141, 220], [221, 300], [301, 380], [381, 453]]

subject = 70

# Determine pointcloud interval for current subject
pointcloud_zip = 'subject_'
for i in pointcloud_subjects:
    if subject >= i[0] and subject <= i[1]:
        pointcloud_zip += '%d_%d.zip' % (i[0], i[1])

for attr in attributes:
    if attr != 'pointcloud':
        url = os.path.join('https://humbi-dataset.s3.amazonaws.com', attr + '_subject', 'subject_%d.zip' % subject)
        path = '%s_subject_%d.zip' % (attr, subject)
        request.urlretrieve(url, path)
        downloaded_zip = zipfile.ZipFile(path)
        downloaded_zip.extractall() # !unzip downloaded_zip
        os.remove(path)

    else:
        pointcloud_url = 'https://humbi-dataset.s3.amazonaws.com/pointcloud/' + pointcloud_zip
        pointcloud_path = 'pointcloud_' + pointcloud_zip
        request.urlretrieve(pointcloud_url, pointcloud_path)
        downloaded_zip = zipfile.ZipFile(pointcloud_path)
        for filename in downloaded_zip.namelist():
            if filename.startswith('subject_%d/' % subject):
                downloaded_zip.extract(filename)
        os.remove(pointcloud_path)

In [ ]:
### Extract mesh vertices and texture for a specific subject and pose

def extract_verts(subject:int, pose:str):
    filename = 'subject_%d/body/%s/reconstruction/smpl_vertex.txt' % (subject, pose)
    verts = torch.Tensor( np.loadtxt(filename) ).to(device).unsqueeze(0)
    return verts


def extract_texture(subject:int, path_to_textures:str):
    filename = 'median_subject_%d.png' % subject
    img_path = os.path.join(path_to_textures, filename)

    image = read_image(img_path)
    image = torch.moveaxis(image, 0, 2).unsqueeze(0).float() * 1.0/255

    return image

In [ ]:
### Construct a pytorch3d meshes object list, optinally with texture

# Returns a list of meshes containing a mesh for each selected poses for a subject

def construct_mesh_list(subject:int, poses:List[str], default_mesh_path:str, path_to_textures:str=None):
    default_mesh = load_obj(default_mesh_path, load_textures=False)
    mesh_faces = default_mesh[1].verts_idx.to(device).unsqueeze(0)

    if path_to_textures is not None:
        verts_uvs = default_mesh[2].verts_uvs.to(device).unsqueeze(0)
        faces_uvs = default_mesh[1].textures_idx.to(device).unsqueeze(0)

    texture_uv = None
    if path_to_textures is not None:
        texture = extract_texture(subject, path_to_textures)

        texture_uv = TexturesUV(maps=texture, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

    meshes = []
    for pose in tqdm(poses):
        mesh_verts = extract_verts(subject, pose)
        mesh = Meshes(mesh_verts, mesh_faces, texture_uv)
        meshes.append(mesh)

    return meshes

In [ ]:
### Construct pytorch3d pointclouds, optinally with normals and surface reconstruction

# Returns a pointcloud with rgb values, another one with normals as color features
# and a reconstructed mesh from a poisson surface reconstruction

def construct_pointcloud(subject:int, pose:str, surface_reconstruction:bool=False):
    filename = 'subject_%d/body/%s/reconstruction/surface_reconstruction.txt' % (subject, pose)

    reconstruction = np.loadtxt(filename)

    # open3d
    #o3d_pointcloud = o3d.geometry.PointCloud()
    #o3d_pointcloud.points = o3d.utility.Vector3dVector(reconstruction[:, :3])

    # pytorch3d
    p3d_points = torch.Tensor(reconstruction[:, :3]).to(device).unsqueeze(0)
    p3d_rgb = torch.Tensor(reconstruction[:, 3:6]).to(device).unsqueeze(0) * 1.0/255

    pointcloud = Pointclouds(points=p3d_points, features=p3d_rgb)

    # surface reconstruction
    if surface_reconstruction:
        # compute normals
        o3d_pointcloud.estimate_normals()
        o3d_pointcloud.orient_normals_consistent_tangent_plane(100)

        p3d_normals = torch.Tensor(o3d_pointcloud.normals).to(device).unsqueeze(0)
        pointcloud_nrm = Pointclouds(points=p3d_points, features=p3d_normals)

        # poisson surface reconstruction in open3d
        o3d_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(o3d_pointcloud, depth=9)

        # remove low density vertices
        densities = np.asarray(densities)
        vertices_to_remove = densities < np.quantile(densities, 0.02)
        o3d_mesh.remove_vertices_by_mask(vertices_to_remove)

        # crop surface outside the bounding box
        bbox = o3d_pointcloud.get_axis_aligned_bounding_box()
        o3d_reconstructed_mesh = o3d_mesh.crop(bbox)

        # pytorch3d reconstruction mesh
        vertices = torch.Tensor(o3d_reconstructed_mesh.vertices).to(device).unsqueeze(0)
        faces = torch.Tensor(o3d_reconstructed_mesh.triangles).to(device).unsqueeze(0)

        reconstructed_mesh = Meshes(vertices, faces)

        return pointcloud, pointcloud_nrm, reconstructed_mesh

    return pointcloud, None, None

In [ ]:
### Construct pointcloud list for all specified poses of a subject

# Returns a list containing the rgb pointclouds for each pose of a subject,
# a second similar list containing pointclouds with normals as color features
# and a list with surface reconstruction for each pose

def pointcloud_list(subject:int, poses:List[str], surface_reconstruction:bool=False):
    pointclouds = []
    normals = []
    reconstructions = []

    if surface_reconstruction:
        print('surface reconstruction can take a while...')

    for pose in tqdm(poses):
        poincloud, normal, reconstruction = construct_pointcloud(subject, pose, surface_reconstruction)
        pointclouds.append(poincloud)
        normals.append(normal)
        reconstructions.append(reconstruction)

    return pointclouds, normals, reconstructions

In [ ]:
### Extract keypoint for specified pose and subject

def extract_keypoint(subject:int, pose:str):
    filename = 'subject_%d/body/%s/reconstruction/keypoints.txt' % (subject, pose)

    keypoints = np.loadtxt(filename)

    return torch.Tensor(keypoints).to(device)

In [ ]:
### Construct keypoint list for all specified poses of a subject

def keypoint_list(subject:int, poses:List[str]):

    keypoints = []
    for pose in poses:
        keypoint = extract_keypoint(subject, pose)
        keypoints.append(keypoint)

    return keypoints

In [ ]:
### Get mapping from OpenPose joints to SMPL vertex indices

def openpose_to_smpl_verts_idx(op_joints_map:dict, vertex_ids:dict):
    correspondances = []

    for key in op_joints_map:
        if key in vertex_ids:
            correspondances.append(vertex_ids[key])
        else:
            correspondances.append(None)

    return correspondances

In [ ]:
### SMPL vertex indices (extracted from OpenPose joints) to vertex positions for a mesh

def keypoints_positions(vertex_correspondances:List[int], mesh:Meshes):
    assert( len(mesh[0].verts_list()) == 1 ), 'mesh can only have one mesh object (one subject, one pose)'

    openpose_keypoints = torch.Tensor().to(device)
    for idx in vertex_correspondances:
        if idx is None:
            pos = [float('nan')] * 3
            pos = torch.Tensor(pos).unsqueeze(0)
        else:
            pos = mesh.verts_padded()[:,idx]
        openpose_keypoints = torch.cat( (openpose_keypoints, pos) )

    return openpose_keypoints

In [ ]:
### Loss for keypoints (euclidian norm of distances between respective keypoints)

def keypoints_loss(gt_keypoints:torch.Tensor, src_keypoints:torch.Tensor):
    assert(gt_keypoints.shape == src_keypoints.shape), 'gt_keypoints and src_keypoints must have the same shape'

    nan_mask = (torch.isnan(gt_keypoints)[:, 0] == False)

    gt_keypoints_masked = gt_keypoints[nan_mask]
    src_keypoints_masked = src_keypoints[nan_mask]

    return torch.linalg.norm(gt_keypoints_masked - src_keypoints_masked)

In [ ]:
### Convert TexturesUV object to TexturesVertex object and convert meshes with TexturesUV to meshes with TexturesVertex functions

# Returns a TexturesVertex object

def convert_to_textureVertex(textures_uv:TexturesUV, meshes:Meshes) -> TexturesVertex:
    verts_colors_packed = torch.zeros_like(meshes.verts_packed()).to(device)
    verts_colors_packed[meshes.faces_packed()] = textures_uv.faces_verts_textures_packed().to(device)

    return TexturesVertex( packed_to_list(verts_colors_packed, meshes.num_verts_per_mesh()) )

# Returns a list of mesh with a TexturesVertex object instead of a TexturesUV object

def convert_mesh_texture(meshes:List[Meshes]):
    # nb_meshes = len(meshes.verts_list())
    assert( isinstance(meshes.textures, TexturesUV) ), 'meshes texture needs to be of type TexturesUV'

    for mesh in meshes:
        verts_features = convert_to_textureVertex(meshes.textures, meshes)
        mesh = Meshes(meshes.verts_list(), meshes.faces_list(), verts_features)

    return meshes

In [ ]:
### Plot interactive scene for multiple meshes or pointclouds

def plot_structure(structures:List[ Union[Meshes,Pointclouds] ]):
    assert(bool(structures) != None), 'nothing to be plotted'

    if not isinstance(structures, list):
        structures_clone = [structures.clone()]
    else:
        structures_clone = []
        for structure in structures:
            structures_clone.append(structure.clone())

    end = len(structures_clone)
    offsets = torch.arange(0, end, step=1)

    dict_string = []
    for i, structure in enumerate(structures_clone):
        offset = torch.Tensor( [offsets[i], 0, 0] ).to(device)
        if isinstance(structure, Meshes):
            structure.verts_list()[0] = structure.verts_list()[0] + offset
            dict_string.append('mesh %d' % (i+1))
        elif isinstance(structure, Pointclouds):
            structure.points_list()[0] = structure.points_list()[0] + offset
            dict_string.append('pointcloud %d' % (i+1))

    zip_iterator = zip(dict_string, structures_clone)
    plot_structures = {'PLOT': dict(zip_iterator)} 

    return plot_scene(plot_structures)

In [ ]:
### Map 25 OpenPose joints (in the order provided by OpenPose) to SMPL joints
# See https://github.com/nkolot/SPIN/blob/master/constants.py
# See https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/02_output.md

op_joints_map = {
    'Nose'     : 24, 'Neck'  : 12, 'RShoulder': 17, 'RElbow'   : 19, 'RWrist' : 21,
    'LShoulder': 16, 'LElbow': 18, 'LWrist'   : 20, 'MidHip'   :  0, 'RHip'   :  2, 
    'RKnee'    :  5, 'RAnkle':  8, 'LHip'     :  1, 'LKnee'    :  4, 'LAnkle' :  7,
    'REye'     : 25, 'LEye'  : 26, 'REar'     : 27, 'LEar'     : 28, 'LBigToe': 29,
    'LSmallToe': 30, 'LHeel' : 31, 'RBigToe'  : 32, 'RSmallToe': 33, 'RHeel'  : 34
}

### SMPL joints to vertices mapping for surface level joints
# See https://github.com/vchoutas/smplx/blob/master/smplx/vertex_ids.py

smpl_vertex_ids = {
    'Nose'   :  332, 'Reye'     : 6260, 'LEye'  : 2800,
    'REar'   : 4071, 'LEar'     :  583, 'RThumb': 6191,
    'RIndex' : 5782, 'RMiddle'  : 5905, 'RRing' : 6016,
    'RPinky' : 6133, 'LThumb'   : 2746, 'LIndex': 2319,
    'LMiddle': 2445, 'LRing'    : 2556, 'LPinky': 2673,
    'LBigToe': 3216, 'LSmallToe': 3226, 'LHeel' : 3387,
    'RBigToe': 6617, 'RSmallToe': 6624, 'RHeel' : 6787
}

In [ ]:
### Construct and plot meshes, pointclouds and keypoints list for chosen subject

subject = 70

poses = []
poses_path = 'subject_%d/body/' % subject
for pose in sorted(os.listdir(poses_path)):
    pose_path = os.path.join(poses_path, pose)
    if os.path.isdir(pose_path):
        poses.append(pose)

obj_mesh = drive_path + 'smpl_bodies/text_uv_coor_smpl.obj'
texture_path = drive_path + 'humbi_maps/humbi_body_texture/body_texture_medians/'


# mesh
try:
    humbi_meshes
except NameError:
    humbi_meshes = construct_mesh_list(subject, poses, default_mesh_path=obj_mesh, path_to_textures=texture_path)

for i, pose in enumerate(poses):
    print('subject {:03d}, pose %s, mesh :'.format(subject) % pose, ':', humbi_meshes[i])

print()


# pointcloud
try:
    humbi_pointclouds
except NameError:
    humbi_pointclouds, humbi_pointclouds_nrm, humbi_reconstructions = pointcloud_list(subject, poses, surface_reconstruction=False)

for i, pose in enumerate(poses):
    print('subject {:03d}, pose %s, rgb pointcloud'.format(subject) % pose, ':'.rjust(4), humbi_pointclouds[i])
    print('subject {:03d}, pose %s, normal pointcloud'.format(subject) % pose, ':'.rjust(0), humbi_pointclouds_nrm[i])

print()


# keypoints
try:
    humbi_keypoints
except NameError:
    humbi_keypoints = keypoint_list(subject, poses)

smpl_verts_correspondances = openpose_to_smpl_verts_idx(op_joints_map, smpl_vertex_ids)

# plotting
plot_structure([humbi_meshes[0], humbi_pointclouds[0]])

In [ ]:
### Construct pointcloud rendered

R, T = look_at_view_transform(20, 10, 0, up=((0, 0, 1),))

cameras = FoVOrthographicCameras(device=device, R=R, T=T).to(device)

raster_settings = PointsRasterizationSettings(
    image_size=512, 
    radius = 0.003,
    points_per_pixel = 10
)

rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)

renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

In [ ]:
### Visualize pointcloud reconstruction with rgb values
rgb_image = renderer(humbi_pointclouds[0]).detach()

plt.figure(figsize=(10, 10))
plt.imshow(rgb_image[0, ..., :3].cpu().numpy())
plt.axis("off")

In [ ]:
### Visualize pointcloud reconstruction with estimated normal values
nrm_image = renderer(humbi_pointcloud_nrm[0]).detach()

plt.figure(figsize=(10, 10))
plt.imshow(nrm_image[0, ..., :3].cpu().numpy())
plt.axis("off")

In [ ]:
### Offset the vertices along their normals
# See https://pytorch3d.readthedocs.io/en/latest/modules/structures.html#pytorch3d.structures.Meshes.offset_verts_

def offset_verts_along_nrm(displace_normals:torch.Tensor, mesh:Meshes):
    verts_packed = mesh.verts_packed()

    if displace_normals.shape != verts_packed[:,0].shape:
        raise ValueError("displace_normals must have dimension (#verts, 3).")

    # update verts packed
    displacement = torch.Tensor().to(device)
    for i in range(3):
        displacement = torch.cat( (displacement, (displace_normals * mesh.verts_normals_packed()[:,i]).unsqueeze(0)) ).to(torch.float32)

    mesh._verts_packed = verts_packed + torch.moveaxis(displacement, 0, 1)
    new_verts_list = list( mesh._verts_packed.split(mesh.num_verts_per_mesh().tolist(), 0) )

    # update verts list
    mesh._verts_list = new_verts_list

    # update verts padded
    if mesh._verts_padded is not None:
        for i, verts in enumerate(new_verts_list):
            if len(verts) > 0:
                mesh._verts_padded[i, : verts.shape[0], :] = verts
    
    # returning mesh itself leads to plotting issues and gradient issues (missing grad_fn problem),
    # we thus construct a new mesh and return it
    return Meshes(mesh.verts_list(), mesh.faces_list(), mesh.textures)

In [ ]:
### Draw n=sample_size random points from pointcloud and construct a new pointcloud from those randomly selected points

def downsample_pointcloud(pointcloud:Pointclouds, sample_size:int):
    shape = pointcloud.num_points_per_cloud().item()
    indices = torch.randperm(shape)[:sample_size]

    points = pointcloud.points_packed()[indices,:].unsqueeze(0)

    normals = None
    features = None

    if pointcloud.normals_packed() is not None:
        normals = pointcloud.normals_packed()[indices,:].unsqueeze(0)

    if pointcloud.features_packed() is not None:
        features = pointcloud.features_packed()[indices,:].unsqueeze(0)

    downsampled_pointcloud = Pointclouds(points, normals, features)

    return downsampled_pointcloud

In [ ]:
### Fit mesh to pointcloud and reconstruction

def fit_mesh(mesh:Meshes, pointcloud:Pointclouds, keypoint:torch.Tensor, displacement:torch.Tensor,
             vertex_correspondances:List[int], iterations:int, optimizer,
             w_chamfer=None, w_edge=None, w_laplace=None, w_normal=None,
             w_pt_edge=None, w_pt_face=None, w_keypoints=None):

    deformed_mesh = mesh.clone().detach()

    loop = tqdm(range(iterations), total = iterations)

    for i in loop:
        optimizer.zero_grad() # initialize optimizer

        # displace
        deformed_mesh = offset_verts_along_nrm(displacement, deformed_mesh.detach())

        loss = 0.0

        pts_to_sample = deformed_mesh.num_verts_per_mesh().item()
        sample_deformed_mesh = sample_points_from_meshes(deformed_mesh, pts_to_sample)

        downsampled_pointcloud = downsample_pointcloud(pointcloud, pts_to_sample)

        if w_chamfer is not None and w_chamfer != 0.0:
            loss_chamfer, _ = chamfer_distance(downsampled_pointcloud, sample_deformed_mesh)
            loss += w_chamfer * loss_chamfer

        if w_edge is not None and w_edge != 0.0:
            loss_edge = mesh_edge_loss(deformed_mesh)
            loss += w_edge * loss_edge

        if w_laplace is not None and w_laplace != 0.0:
            loss_laplacian = mesh_laplacian_smoothing(deformed_mesh, method="uniform")
            loss += w_laplace * loss_laplacian

        if w_normal is not None and w_normal != 0.0:
            loss_normal = mesh_normal_consistency(deformed_mesh)
            loss += w_normal * loss_normal

        if w_pt_edge is not None and w_pt_edge != 0.0:
            loss_pt_edge = point_mesh_edge_distance(deformed_mesh, downsampled_pointcloud)
            loss += w_pt_edge * loss_pt_edge

        if w_pt_face is not None and w_pt_face != 0.0:
            loss_pt_face = point_mesh_face_distance(deformed_mesh, downsampled_pointcloud)
            loss += w_pt_face * loss_pt_face

        '''
        if w_norm is not None and w_pt_face != 0.0:
            loss_norm = torch.linalg.norm(displacement)
            loss += w_norm * loss_norm
        '''

        if w_keypoints is not None and w_pt_face != 0.0:
            gt_keypoints = keypoints_positions(vertex_correspondances, deformed_mesh)
            loss_keypoints = keypoints_loss(gt_keypoints, keypoint)
            loss += w_keypoints * loss_keypoints
        
        loop.set_description('total_loss = %.6f' % loss)
        loss.backward()
        optimizer.step()

    return displacement, loss

In [ ]:
def fit_meshes(meshes:List[Meshes], pointclouds:List[Pointclouds], keypoints:List[torch.Tensor],
               vertex_correspondances:List[int], iterations_per_subject:int,
               w_norm=0.0, w_chamfer=None, w_edge=None, w_laplace=None,
               w_normal=None, w_pt_edge=None, w_pt_face=None, w_keypoints=None):

    assert(len(meshes) == len(pointclouds)), 'meshes and pointclouds lists should be of same length'

    displace_normals = torch.full([meshes[0].num_verts_per_mesh()], 0.0, device=device, requires_grad=True)

    optimizer = torch.optim.Adam([displace_normals], lr=0.01, weight_decay=w_norm)

    displacements = []
    losses = []

    for i, (mesh, pointcloud, keypoint) in enumerate(zip(meshes, pointclouds, keypoints)):
        print( 'fit mesh %d out of %d' % (i+1, len(meshes)) )
        displace_normals, loss = fit_mesh(mesh, pointcloud, keypoint, displace_normals, vertex_correspondances, iterations_per_subject, optimizer, w_chamfer, w_edge, w_laplace, w_normal, w_pt_edge, w_pt_face, w_keypoints)
        displacements.append(displace_normals)
        losses.append(loss)

    return displacements, losses

In [ ]:
### Fit humbi_meshes to humbi_pointclouds for considered subject
fitted_displacements, losses = fit_meshes(humbi_meshes[0], humbi_pointclouds[0], humbi_keypoints,
                                          smpl_verts_correspondances, iterations_per_subject=1000,
                                          w_norm=1.0, w_chamfer=1.0, w_edge=1.0, w_laplace=0.1,
                                          w_normal=0.1, w_pt_edge=0.0, w_pt_face=0.0, w_keypoints=0.0)

In [ ]:
### Find fitted mesh with lowest error and plot the corresponding displacements onto first mesh

index = 0 # plot pose with index
min_idx = np.argmin(losses) # pose with minimal loss

deformed_mesh = humbi_meshes[index].clone()
deformed_mesh = offset_verts_along_nrm(fitted_displacements[min_idx], deformed_mesh)

plot_structure([humbi_meshes[index], deformed_mesh, humbi_pointclouds[index]])

In [ ]:
### BPS

need_bps=False
try:
    import bps
except ModuleNotFoundError:
    need_bps=True

if need_bps:
    !pip install git+https://github.com/sergeyprokudin/bps


from bps import bps


# Average displacements
def average_displacements(meshes:List[Meshes], pointclouds:List[Pointclouds]):

    avg_displ = torch.Tensor().to(device)

    for mesh, pointcloud in zip(meshes, pointclouds):
        displ = displace_verts(mesh.clone(), pointcloud.clone())
        avg_displ = torch.cat((avg_displ, displ.unsqueeze(0)))

    return torch.mean(avg_displ, dim=0)


# Displace each vertex from a subject by lambda, where lambda = projection of vector to closest point to normal vector of considered vertex
def displace_verts(mesh:Meshes, pointcloud:Pointclouds):
    points = pointcloud.points_packed().unsqueeze(0).detach().cpu().numpy()

    verts = mesh.verts_packed().detach().cpu().numpy()
    vert_normals = mesh.verts_normals_packed()

    displacements = bps.encode(points, bps_arrangement='custom', custom_basis=verts, bps_cell_type='deltas')
    displacements = torch.Tensor(displacements).squeeze().to(device)

    displacement_along_nrm = torch.sum(displacements * vert_normals, dim=1).to(device)

    return displacement_along_nrm


# Test displacements along normals with nearest points in pointcloud (bps)
try:
    avg_displacements
except NameError:
    avg_displacements = average_displacements(humbi_meshes, humbi_pointclouds)

index = 0
deformed_mesh = humbi_meshes[index].clone()
deformed_mesh = offset_verts_along_nrm(avg_displacements, deformed_mesh)

plot_structure(deformed_mesh)

In [ ]:
### Extract vertex uv pixel positions on a 2D square map
# See https://github.com/facebookresearch/pytorch3d/discussions/588

def verts_uvs_positions(nb_verts:int, default_mesh_path:str, map_size:int=1024):
    default_mesh = load_obj(default_mesh_path, load_textures=False)

    flatten_verts_idx = default_mesh[1].verts_idx.flatten()
    flatten_textures_idx = default_mesh[1].textures_idx.flatten()
    verts_uvs = default_mesh[2].verts_uvs

    verts_to_uv_index = torch.zeros(nb_verts, dtype=torch.int64)
    verts_to_uv_index[flatten_verts_idx] = flatten_textures_idx
    verts_to_uvs = verts_uvs[verts_to_uv_index]

    uv_x = ( float(map_size) * verts_to_uvs[:,0] ).unsqueeze(0)
    uv_y = ( float(map_size) * (1.0 - verts_to_uvs[:,1]) ).unsqueeze(0)
    verts_uvs_positions = torch.cat((uv_x, uv_y)).moveaxis(0,1).round()

    return verts_uvs_positions

In [ ]:
### Create displacement map for each vertex and perform interpolation (inpainint) between vertex values

def inpainted_displacements(subject:int, displacements:torch.Tensor, verts_uvs_positions:torch.Tensor, path_to_textures:str=None,
                            map_size:int=1024, inpainting_method='telea', return_texture:bool=False):

    assert(inpainting_method == 'telea' or inpainting_method == 'ns'), "inpainting_method must be one of 'telea' or 'ns' (Navier-Stokes)"
    
    displacement_map = torch.full((map_size,map_size), 255.0/2.0, dtype=torch.uint8, device=device)
    inpaint_mask = torch.ones((map_size,map_size), dtype=torch.uint8)

    texture = read_image(path_to_textures + 'median_subject_%d.png' % subject)
    texture = torch.moveaxis(texture, 0, 2)

    mask_condition = (texture[:,:,0] == 0) & (texture[:,:,1] == 0) & (texture[:,:,2] == 0)
    inpaint_mask[mask_condition] = 0

    displacements_uint = displacements
    if displacements.type() != 'torch.ByteTensor':
        displacements_uint = (displacements * 255).round().type(torch.uint8)

    for disp, pos in zip(displacements_uint, verts_uvs_positions):
        pos_x = int( pos[0].item() )
        pos_y = int( pos[1].item() )
        displacement_map[pos_y, pos_x] = disp # pixels in our constructed displacement map which get a value
        inpaint_mask[pos_y, pos_x] = 0 # pixels that are not inpainted

    displacement_map[mask_condition] = 255.0/2.0

    # inpainting
    method = cv2.INPAINT_TELEA * (inpainting_method=='telea') + cv2.INPAINT_NS * (inpainting_method=='ns')
    inpainted_displacements = cv2.inpaint(displacement_map.numpy(), inpaint_mask.numpy(), 3, method)

    if return_texture:
        return torch.Tensor(inpainted_displacements), displacement_map, inpaint_mask, texture
    else:
        return torch.Tensor(inpainted_displacements), displacement_map, inpaint_mask, None

In [ ]:
### Test displacements inpainting

nb_verts = fitted_displacements[min_idx].size(dim=0)
verts_uvs = verts_uvs_positions(nb_verts, obj_mesh)

inpainted_displacements, displacement_map, inpaint_mask, texture = inpainted_displacements(subject, fitted_displacements[min_idx], verts_uvs, texture_path, return_texture=True)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(texture)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(inpaint_mask, cmap='gray')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(displacement_map, cmap='gray')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(inpainted_displacements, cmap='gray')